# Setup

In [1]:
import json
import pprint
import numpy as np
from matplotlib import pyplot as plt
import os
from pathlib import Path
from utilities_network import data_dict_to_2d_array, data_dict_to_3d_array, scale_data_array
from utilities_network import get_time_networks, plot_heatmaps
import utilities_visualization

class InterSystemRecurrenceNetwork(InteractingNetworks):

    """
    Generating and quantitatively analyzing inter-system recurrence networks.

    For a inter-system recurrence network, time series 


# Data

Active slice (here, [1300,2300]) of each quantity of each sensor.

- Inititally stored as a dictionary.
- First level: 5 sensors ('OR', 'OL', 'IB', 'IT', 'IL')
- Second level: 22 quantities ('time', 'ax', 'ay', 'az', 'phi', 'theta', 'psi', 'wx', 'wy', 'wz', 'phidot', 'thetadot', 'psidot', 'Ax', 'Ay', 'Az', 'dx', 'dy', 'dz', 'vx', 'vy', 'vz')

In [2]:
SCALE_DATA_OVERALL = False # whether to scale data array all at once (overall), or individually by time series
USE_THRESHOLDS = True # whether to use distance threshold instead of recurrence rate

In [3]:
# Data was saved as a dictionary using experiment_save_data.py.
# To regenerate the data (use different experiment data or different active slice other than [1300-2300]),
# Go to experiments directory and run in python:
    # python experiment_save_data.py

with open("../../experiments/data_2025-09-03 Euler Angles_20250903_203926.json", 'r') as f:
    data_dict = json.load(f)

# Directory to get windowed network results
if SCALE_DATA_OVERALL:
    DATA_DIR = Path("network_time_plots/scaled_overall")
else:
    DATA_DIR = Path("network_time_plots/scaled_individually")
if USE_THRESHOLDS:
    DATA_DIR = DATA_DIR/"th"
else:
    DATA_DIR = DATA_DIR/"rr"
# Load windowed network results
network_windows = np.load(DATA_DIR/'network_windows.npz',allow_pickle=True)['network_windows'].item()

# Directory to save heatmaps
if SCALE_DATA_OVERALL:
    OUT_DIR = Path("heatmaps/scaled_overall")
else:
    OUT_DIR = Path("heatmaps/scaled_individually")
if USE_THRESHOLDS:
    OUT_DIR = OUT_DIR/"th"
else:
    OUT_DIR = OUT_DIR/"rr"

if not OUT_DIR.exists():
    os.makedirs(OUT_DIR)

# Plot Network Heatmaps

In [4]:
# To plot new heatmaps
PLOT_NEW = True

In [5]:
# Number of timesteps to analyze (first `N_TIME` timesteps will be used)
N_TIME = 1000

network_parameters = {
    'angular_velocities_with_time': dict(
        sensors=['OR','IT','IL','IB'],
        quantities=[['wx','time'],['wy','time'],['wz','time']],
        heatmap_target_quantities=['wz'],
        heatmap_max=0.4,
    ),
    'euler_phase_space_with_time': dict(
        sensors=['OR','IT','IL','IB'],
        quantities=[['phi','phidot','time'],['theta','thetadot','time'],['psi','psidot','time']],
        heatmap_target_quantities=['psi'],
        heatmap_max=0.05,
    ),
    'linear_phase_space_with_time': dict(
        sensors=['OR','IT','IL','IB'],
        quantities=[['dx','vx','time'],['dy','vy','time'],['dz','vz','time']],
        heatmap_target_quantities=['dz'],
        heatmap_max=0.05,
    ),
    # 'angular_velocity_3d_with_time': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['wx', 'wy', 'wz', 'time']],
    # ),
    # 'euler_angle_3d_with_time': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['phi', 'theta', 'psi', 'time']],
    # ),
    # 'linear_displ_3d_with_time': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['dx', 'dy', 'dz', 'time']],
    # ),
    # 'angular_velocities': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['wx'],['wy'],['wz']],
    #     heatmap_target_quantities=['wz'],
    # ),
    # 'euler_phase_space': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['phi','phidot'],['theta','thetadot'],['psi','psidot']],
    #     heatmap_target_quantities=['psi'],
    # ),
    # 'linear_phase_space': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['dx','vx'],['dy','vy'],['dz','vz']],
    #     heatmap_target_quantities=['dz'],
    # ),
    # 'angular_velocity_3d': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['wx', 'wy', 'wz']],
    # ),
    # 'euler_angle_3d': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['phi', 'theta', 'psi']],
    # ),
    # 'linear_displ_3d': dict(
    #     sensors=['OR','IT','IL','IB'],
    #     quantities=[['dx', 'dy', 'dz']],
    # ),
}

In [6]:
mappings = {}
target_node_lists = {}

sensor_name_mapping = {
    'OR': 'h',
    'IB': 'f',
    'IT': 't',
    'IL': 'c'
}

quantity_name_mapping = {
    'wx': r"$\omega_{x}$",
    'wy': r"$\omega_{y}$",
    'wz': r"$\omega_{z}$",
    'dx': r"$d_{x}$",
    'dy': r"$d_{y}$",
    'dz': r"$d_{z}$",
    'vx': r"$v_{x}$",
    'vy': r"$v_{y}$",
    'vz': r"$v_{z}$",
    'phi': r"$\phi$",
    'theta': r"$\theta$",
    'psi': r"$\psi$",
    'phidot': r"$\dot{\phi}$",
    'thetadot': r"$\dot{\theta}$",
    'psidot': r"$\dot{\psi}$",
}

def get_mappings(**config):
    sensors = config['sensors']
    quantities = config['quantities']
    nq = len(quantities)
    mapping = {
        nq*i+j: f"{sensor_name_mapping[s]}{quantity_name_mapping[q[0]]}"
        for i,s in enumerate(sensors)
        for j,q in enumerate(quantities)
    }
    return mapping

def get_target_nodes(**config):
    quantities = config['quantities']
    target_nodes = [f"{sensor_name_mapping['OR']}{quantity_name_mapping[q[0]]}"
        for q in quantities
    ]
    return target_nodes

for title,parameters in network_parameters.items():
    mappings[title] = get_mappings(**parameters)
    target_node_lists[title] = get_target_nodes(**parameters)

In [7]:
if PLOT_NEW:
    for title,parameters in network_parameters.items():
        plot_heatmaps(network_windows[title],
                mappings[title],
                target_node_lists[title],
                heatmap_target_quantities=parameters.get('heatmap_target_quantities',None),
                width_scale=1.0,
                heatmap_max=parameters.get('heatmap_max',0.25),
                time=data_dict['OR']['time'],
                plot_filename_prefix=OUT_DIR/title)

Saving heatmaps/scaled_individually/th/angular_velocities_with_time/to_hoop/C_xys.eps
Saving heatmaps/scaled_individually/th/angular_velocities_with_time/to_hoop/C_xys.png
Saving heatmaps/scaled_individually/th/angular_velocities_with_time/to_hoop/T_xys.eps
Saving heatmaps/scaled_individually/th/angular_velocities_with_time/to_hoop/T_xys.png
Saving heatmaps/scaled_individually/th/angular_velocities_with_time/from_hoop/C_xys.eps
Saving heatmaps/scaled_individually/th/angular_velocities_with_time/from_hoop/C_xys.png
Saving heatmaps/scaled_individually/th/angular_velocities_with_time/from_hoop/T_xys.eps
Saving heatmaps/scaled_individually/th/angular_velocities_with_time/from_hoop/T_xys.png
Saving heatmaps/scaled_individually/th/euler_phase_space_with_time/to_hoop/C_xys.eps
Saving heatmaps/scaled_individually/th/euler_phase_space_with_time/to_hoop/C_xys.png
Saving heatmaps/scaled_individually/th/euler_phase_space_with_time/to_hoop/T_xys.eps
Saving heatmaps/scaled_individually/th/euler_phas